# Pasos de ETL
### 1) Lectura de data Externa (GPD 2023 por estado, Estados con respectivas ciudades ,Ciudades más visitadas en el 2022)
### 2) Lectura de datos de Google Maps

***Lectura de librerías***

In [3]:
import pandas as pd

## 1) Lectura de Data Externa

***
- Base de datos con los 51 estados con Mayor GDP en millones de USD, filtrar 10 primeras 
- Base de datos con todas las ciudades y estados
- Base de datos del top 10 de ciudades más turísticas en personas, filtrar 5 primeras
***


In [4]:
# Lectura de GDP per capita por los 51 estados de USA
archivoGDP = "GDPPerCapita.xlsx"
dfGDPporEstado = pd.read_excel(archivoGDP)
# Ordenar de mayor a menor por estado según GDP 2023
dfGDPporEstadoordenado = dfGDPporEstado.sort_values(by=2023, ascending=False)
# Elegir los primero 10 estados
top10EstadosMayorGDP = dfGDPporEstadoordenado.head(10).reset_index(drop=True)
# Quitar a los estados, el espacio al final de cada fila y elimiar columna 2022
top10EstadosMayorGDP['State'] = top10EstadosMayorGDP['State'].str.strip()
top10EstadosMayorGDP = top10EstadosMayorGDP.drop(columns=[2022])
top10EstadosMayorGDP

,State,2023
0,California,3233151
1,Texas,2032933
2,New York,1775714
3,Florida,1279119
4,Illinois,875569
5,Pennsylvania,789502
6,Ohio,698217
7,Washington,672125
8,Georgia,661115
9,New Jersey,656480


In [5]:
# Lectura de archivos de todas las ciudades por estado
citiesPerState = "uscities.csv"
# Lee el archivo CSV con el separador ;
ciudadesYEstados = pd.read_csv(citiesPerState, sep=';')
# Cambiar nombre de State_name por State
ciudadesYEstados.rename(columns={'state_name': 'State'}, inplace=True)
#Unir State-city para hacer el posterior Merge
ciudadesYEstados['State-City'] = ciudadesYEstados['State'] + '-' + ciudadesYEstados['city']
ciudadesYEstados

,city,city_ascii,state_id,State,State-City
0,New York,New York,NY,New York,New York-New York
1,Los Angeles,Los Angeles,CA,California,California-Los Angeles
2,Chicago,Chicago,IL,Illinois,Illinois-Chicago
3,Miami,Miami,FL,Florida,Florida-Miami
4,Houston,Houston,TX,Texas,Texas-Houston
...,...,...,...,...,...
31115,Kohatk,Kohatk,AZ,Arizona,Arizona-Kohatk
31116,Ironville,Ironville,PA,Pennsylvania,Pennsylvania-Ironville
31117,Newkirk,Newkirk,NM,New Mexico,New Mexico-Newkirk
31118,Falcon Village,Falcon Village,TX,Texas,Texas-Falcon Village


In [6]:
# Top 10 Ciudades con más visitantes en el año 2022
# Definición de nombre de archivo
archivoTop10Cities = "Top10MostVisitedCities.xlsx"
# Lectura de archivo de excel con el top 10 de ciudades más visitadas en USA
Top10Cities = pd.read_excel(archivoTop10Cities)
# Renombrar la columna City, por city, para merge posterior
Top10Cities.rename(columns={'City': 'city'}, inplace=True)
#Unir State-city para hacer el posterior Merge
Top10Cities['State-City'] = Top10Cities['State'] + '-' + Top10Cities['city']
Top10Cities

,Rank,city,2022 Visitation,State,State-City
0,1,New York,6992000,New York,New York-New York
1,2,Miami,4086000,Florida,Florida-Miami
2,3,Orlando,2920000,Florida,Florida-Orlando
3,4,Los Angeles,2750000,California,California-Los Angeles
4,5,San Francisco,1739000,California,California-San Francisco
5,6,Las Vegas,1660000,Nevada,Nevada-Las Vegas
6,7,Washington,1167000,Washington,Washington-Washington
7,8,Chicago,1061000,Illinois,Illinois-Chicago
8,9,Boston,738000,Massachusetts,Massachusetts-Boston
9,10,Honolulu,711000,Hawaii,Hawaii-Honolulu


In [7]:
# Combinación de datos, top 10 GDP percapita y tabla de ciudades y estados 
merge1GDPYCiudadesYestados = pd.merge(top10EstadosMayorGDP, ciudadesYEstados, on='State', how='inner')
# COmbinación con el top 10 ciudades más turisticas
merge2TopCitiesYGDPYCiudadesYestados = pd.merge(Top10Cities, merge1GDPYCiudadesYestados, on='State-City', how='inner')
# Ordenar por 2022 Visitation y agarrar los 5 primeros
merge2TopCitiesYGDPYCiudadesYestados = merge2TopCitiesYGDPYCiudadesYestados.sort_values(by='2022 Visitation', ascending=False).head(5)
merge2TopCitiesYGDPYCiudadesYestados
# Eliminar columnas no importantes
merge2TopCitiesYGDPYCiudadesYestados = merge2TopCitiesYGDPYCiudadesYestados.drop(columns=['State_y','city_y'])
# Poner nuevos nombres
estadosYCiudadesAUSAR = merge2TopCitiesYGDPYCiudadesYestados.rename(columns={'city_x': 'city', 'State_x': 'State',2023 : 'GDPporEstado2023'})

estadosYCiudadesAUSAR

,Rank,city,2022 Visitation,State,State-City,GDPporEstado2023,city_ascii,state_id
0,1,New York,6992000,New York,New York-New York,1775714,New York,NY
1,2,Miami,4086000,Florida,Florida-Miami,1279119,Miami,FL
2,3,Orlando,2920000,Florida,Florida-Orlando,1279119,Orlando,FL
3,4,Los Angeles,2750000,California,California-Los Angeles,3233151,Los Angeles,CA
4,5,San Francisco,1739000,California,California-San Francisco,3233151,San Francisco,CA


***
Solo Se elige para revisión
1) New York
2) Florida
3) California
***

## 2) Lectura de archivos Google Maps

### 2.1) Lectura review de New York

In [8]:
file_names = ["1.json", "2.json", "3.json", "4.json", "5.json", "6.json", "7.json", "8.json", "9.json", "10.json", "11.json", "12.json", "13.json", "14.json", "15.json", "16.json", "17.json", "18.json"]
dfNewYork = []
for file_name in file_names:
    json_file_path = f"review-New_York/{file_name}"
    df = pd.read_json(json_file_path, lines=True,dtype=str)
    dfNewYork.append(df)
dfNewYorkReviews = pd.concat(dfNewYork, ignore_index=True)

In [9]:
dfNewYorkReviews.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
392668,109892647268396416250,benny w,1524432497469,5,Thanks Daniel! Had a amazing experience. Very ...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c24c4f8dd42815:0x468f9c773490e12f
2642261,115018989612279544448,Ravi Rajesh Fernando,1572813435438,5,None,None,None,0x89e831e7ee7af91f:0xa829b4b2bd5a60da
265869,117480564369654646399,Nick A,1469975370463,3,None,None,None,0x89c25f67ebb66dc5:0x14d546c7d1570130


### 2.2) Lectura review Florida

In [10]:
file_names = ["1.json", "2.json", "3.json", "4.json", "5.json", "6.json", "7.json", "8.json", "9.json", "10.json", "11.json", "12.json", "13.json", "14.json", "15.json", "16.json", "17.json", "18.json"]
dfFlorida = []

for file_name in file_names:
    json_file_path = f"review-Florida/{file_name}"
    df = pd.read_json(json_file_path, lines=True,dtype=str)
    dfFlorida.append(df)
dfFloridaReviews = pd.concat(dfFlorida, ignore_index=True)

In [11]:
dfFloridaReviews.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
1095237,107777713413152770358,Toussaint Lewis,1609265610699,5,"Lexus/Toyota West Kendall. Knowledgeable, Prof...",None,"{'time': 1609269816505, 'text': 'Thank you for...",0x88d9c1837c249c51:0xe202fa132024bc8d
1967829,113476404919656666300,Colin Keeler,1499792490372,5,Great customer service no lines no waiting mad...,None,None,0x88d9ac9d69438b39:0xdaad9cb868bdb4f6
2654611,108806575786826153481,Paula Freed,1557550981515,5,This place is so much fun! I really had a grea...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1557760223251, 'text': 'Thank you for...",0x88e77e4cd5668607:0x230dfa52bcd29398


### 2.3) Lectura review California

In [12]:
file_names = ["1.json", "2.json", "3.json", "4.json", "5.json", "6.json", "7.json", "8.json", "9.json", "10.json", "11.json", "12.json", "13.json", "14.json", "15.json", "16.json", "17.json", "18.json"]
dfCalifornia = []

for file_name in file_names:
    json_file_path = f"review-California/{file_name}"
    df = pd.read_json(json_file_path, lines=True,dtype=str)
    dfCalifornia.append(df)
dfCaliforniaReviews = pd.concat(dfCalifornia, ignore_index=True)

In [13]:
dfCaliforniaReviews.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
1891951,111690357900736526804,Ronnie Salas,1615053050005,5,None,None,"{'time': 1615131355183, 'text': 'Ronnie,\nThan...",0x80dd32f142b8252b:0x1af197c9399f5231
1555057,103501971486645700350,Patsy Rivera,1591422230397,5,Love watching my car there.,None,None,0x80c2d19062082955:0x58899c4e0cd9ea17
1923105,107283960074800329227,Alan García,1555724942643,1,I've been meaning to add a review for a long t...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x80deab2e6b0a2f2b:0x757d25321229314b


### Concatenar todas la Tablas

In [14]:
dfReviewCaNyFlTotal = pd.concat([dfNewYorkReviews, dfFloridaReviews, dfCaliforniaReviews], ignore_index=True)
#Dropeamos las columnas que no usaremos
dfReviewCaNyFlTotal = dfReviewCaNyFlTotal.drop(["name", "pics", "resp","time","user_id"], axis=1)
# Dejamos texto, para hacer el análisis de sentimiento

In [15]:
dfReviewCaNyFlTotal.sample(3)

,rating,text,gmap_id
5762969,5,This place is amazing! My boyfriends son goes ...,0x80ec5e9637fb8c03:0x9499923c1814c02b
6041570,5,❤,0x80c2a023128db849:0x52a470ebe1cb6b83
4640888,3,None,0x88dd81eb70d3168f:0x7ef2d56737e63fa2


### 2.4) Lectura de Metasitios

In [16]:
file_names = ["1.json", "2.json", "3.json", "4.json", "5.json", "6.json", "7.json", "8.json", "9.json", "10.json", "11.json"]
dfMetasitiosGoogle = []

for file_name in file_names:
    json_file_path = f"metadata-sitios/{file_name}"
    df = pd.read_json(json_file_path, lines=True)
    dfMetasitiosGoogle.append(df)
dfMetasitiosTotal = pd.concat(dfMetasitiosGoogle, ignore_index=True)

### Filtro de Metasitios por Restaurant

In [17]:
# Explorar Sitios
dfMetasitiosTotal = df.explode('category')
# Filtrar por los que tienen Restaurante 
df_MetasitiosRestaurante = dfMetasitiosTotal[dfMetasitiosTotal['category'] == 'Restaurant']

In [18]:
df_MetasitiosRestaurante.sample(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
79354,Hosmer City Liquor,"Hosmer City Liquor, 101 E Main St, Hosmer, SD ...",0x52d3f5dcef20e9ad:0xcd047ca6c8f55de8,None,45.578611,-99.475000,Restaurant,4.6,18,None,"[[Wednesday, 4–10PM], [Thursday, 4–10PM], [Fri...","{'Service options': ['Curbside pickup', 'No-co...",Closed ⋅ Opens 4PM,"[0x52d3f5dc5e385cd5:0xc7001c1e05d5471a, 0x52d3...",https://www.google.com/maps/place//data=!4m2!3...
208997,Subway,"Subway, 5920 Roswell Rd NE Suite B105, Atlanta...",0x88f50e937515926d:0xe19f98f6a8ea5793,Casual counter-serve chain for build-your-own ...,33.916321,-84.379801,Restaurant,3.8,58,$,"[[Tuesday, 8AM–7PM], [Wednesday, 8AM–7PM], [Th...","{'Service options': ['Curbside pickup', 'No-co...",Closed ⋅ Opens 8AM,"[0x88f50e96a4fa12df:0x65780735b44a1d98, 0x88f5...",https://www.google.com/maps/place//data=!4m2!3...
37715,King Pizza,"King Pizza, 3522, 1140 Memorial Ave, West Spri...",0x89e6e134e77a698d:0x346a915c93169e6f,None,42.094015,-72.616639,Restaurant,3.3,38,None,"[[Wednesday, 11AM–11PM], [Thursday, 11AM–11PM]...","{'Service options': ['Takeout', 'Delivery']}",Permanently closed,"[0x89e6e12b08e57687:0x433b680c601494ff, 0x89e6...",https://www.google.com/maps/place//data=!4m2!3...


### Union de Reviews Google con Metasitios

In [19]:
# Unión de DataFrames Google
googleDfs = pd.merge(dfReviewCaNyFlTotal, df_MetasitiosRestaurante, on='gmap_id', how='inner')
# Eliminar Filas que no Importan
googleDfs = googleDfs.drop(["url", "relative_results", "MISC","price","description","state","hours"], axis=1)

# Convertir a float el rating
googleDfs['rating'] = pd.to_numeric(googleDfs['rating'], errors='coerce', downcast='integer')
googleDfs.sample(3)

,rating,text,gmap_id,name,address,latitude,longitude,category,avg_rating,num_of_reviews
774,5,None,0x89c25f5ec58a10f7:0x365d64ed71656b09,Samossa Bites,"Samossa Bites, 3527 31st St, Queens, NY 11106",40.757222,-73.928889,Restaurant,4.2,105
18575,4,None,0x89d374c581ca43c7:0xe9c07e4185585a94,Max's Classic American Grill,"Max's Classic American Grill, 4600 Genesee St,...",42.935137,-78.716076,Restaurant,3.7,198
22545,3,None,0x89d9f31b319b591b:0x20e3403dfc02ebba,Joe's To Go,"Joe's To Go, 415 W Onondaga St, Syracuse, NY 1...",43.040493,-76.157139,Restaurant,4.6,98


In [20]:
# Exportar CSV
googleDfs.to_csv('datos_googleMaps.csv', index=False)